# EasyPandas: a simple package for learning about data transformations in Python

05 01 25

[Pandas](https://pandas.pydata.org/) is the most common package for carrying out data analysis in Python, but it can involve some esoteric syntax. Here we use a simplified package called EasyPandas for the purposes of introducing the main operations that data analysts carry out on data.

## Data transformation operations
In this notebook we demonstrate the main operations used by data analysts through some simple examples.

The operations are:
- **filter**: filters the dataframe to a subset of rows that meet a particular condition
- **select**: select a subset of columns
- **sort**: order the dataframe based on the values in a specific column
- **groupby**: split the dataframe into groups, apply an aggregate function to a column within each group and then combine
- **add_column**: add a column to the dataframe with a specific set of values
- **add_derived_column**: add a column to the dataframe with values derived from another column
- **join**: join one dataframe to another

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from easypandas import EasyDataFrame
from d2k_utils_2025 import get_month, get_year, get_day_of_week

sns.set()
%matplotlib inline

## Create some data

First we need some data to work with. 

Let's create the same table that is described in the data transformations section of the course materials.

In [2]:
# fictitious blood test results
df = EasyDataFrame(data={'patientid': [1, 1, 1, 2, 2, 3, 3], 
                         'date': ['24-09-2013', '15-11-2013', '06-06-2014', '01-09-2016', '06-09-2016', '31-08-2000', '29-08-2001'], 
                         'blood_test_name': ['Haemoglobin', 'Haemoglobin', 'Platelets', 'Haemoglobin', 'Haemoglobin', 'Platelets', 'WCC'],
                         'blood_test_result': [154, 149, 226, 132, 139, 250, 6.2]})

# convert dates to datetime values
df = df.assign(date=lambda df_: pd.to_datetime(df_.date, dayfirst=True))

# patient postcode information
df2 = EasyDataFrame(data={'patientid': [1, 2, 3], 'postcode': [3000, 3078, 3056]})

In [3]:
df2

,patientid,postcode
0,1,3000
1,2,3078
2,3,3056


In [4]:
df

,patientid,date,blood_test_name,blood_test_result
0,1,2013-09-24,Haemoglobin,154.0
1,1,2013-11-15,Haemoglobin,149.0
2,1,2014-06-06,Platelets,226.0
3,2,2016-09-01,Haemoglobin,132.0
4,2,2016-09-06,Haemoglobin,139.0
5,3,2000-08-31,Platelets,250.0
6,3,2001-08-29,WCC,6.2


## Carry out some operations

Now we can try out some of the operations that are described in the course materials

### filter

This operation selects only those rows of the table that meet certain conditions.

In the first example we select only those rows where the `blood_test_name` is equal to `Haemoglobin`.

In [5]:
df.filter('blood_test_name', 'equals', 'Haemoglobin')

,patientid,date,blood_test_name,blood_test_result
0,1,2013-09-24,Haemoglobin,154.0
1,1,2013-11-15,Haemoglobin,149.0
3,2,2016-09-01,Haemoglobin,132.0
4,2,2016-09-06,Haemoglobin,139.0


In the second example we only select rows where `blood_test_name` is equal to `WCC` (White Cell Count)

In [6]:
df.filter('blood_test_name', 'equals', 'WCC')

,patientid,date,blood_test_name,blood_test_result
6,3,2001-08-29,WCC,6.2


### select

This operation selects a subset of columns of the original table, in this case `blood_test_name` and `blood_test_result`.

In [7]:
df.select(['blood_test_name', 'blood_test_result'])

,blood_test_name,blood_test_result
0,Haemoglobin,154.0
1,Haemoglobin,149.0
2,Platelets,226.0
3,Haemoglobin,132.0
4,Haemoglobin,139.0
5,Platelets,250.0
6,WCC,6.2


### groupby

The groupby operation has three elements: 
1) The columns used to separate the original table into groups;
2) The column that we want to carry out a calculation on; and 
3) An operation to carry out

The operation below first divides the table into grouped defined by the value of the `blood_test_name`. Then for each group it calculates the maximum value of `blood_test_result`.

In this case, we can think of the table as being split into three different values of `blood_test_name`: *Haemoglobin*, *Platelets* and *WCC*. Within each of these groups we then find the row with the highest value of `blood_test_result`.

In [8]:
df.groupby('blood_test_name', 'blood_test_result', 'max')

,blood_test_name,blood_test_result
0,Haemoglobin,154.0
1,Platelets,250.0
2,WCC,6.2


### sort

A simpler operation, **sort**, involves re-ordering the values of the original table based on each row's value from a particular column. 

In the example below the rows are ordered from oldest to most recent based on the value in the `date` column.

In [9]:
df.sort('date', 'ascending')

,patientid,date,blood_test_name,blood_test_result
5,3,2000-08-31,Platelets,250.0
6,3,2001-08-29,WCC,6.2
0,1,2013-09-24,Haemoglobin,154.0
1,1,2013-11-15,Haemoglobin,149.0
2,1,2014-06-06,Platelets,226.0
3,2,2016-09-01,Haemoglobin,132.0
4,2,2016-09-06,Haemoglobin,139.0


### add_column

We can also add additional columns to our table and set the value of the column. In the example below we create a `table_number` column with value 1.

In [10]:
df.add_column('table_number', 1)

,patientid,date,blood_test_name,blood_test_result,table_number
0,1,2013-09-24,Haemoglobin,154.0,1
1,1,2013-11-15,Haemoglobin,149.0,1
2,1,2014-06-06,Platelets,226.0,1
3,2,2016-09-01,Haemoglobin,132.0,1
4,2,2016-09-06,Haemoglobin,139.0,1
5,3,2000-08-31,Platelets,250.0,1
6,3,2001-08-29,WCC,6.2,1


### add_derived_column

We can also create a new column based on values from another column. In this case we create a column `month_name` by extracting the month from the `date` column. We have to specify the `get_month` function that will do this extraction.

In [11]:
df.add_derived_column('date', 'month_name', get_month)

,patientid,date,blood_test_name,blood_test_result,month_name
0,1,2013-09-24,Haemoglobin,154.0,September
1,1,2013-11-15,Haemoglobin,149.0,November
2,1,2014-06-06,Platelets,226.0,June
3,2,2016-09-01,Haemoglobin,132.0,September
4,2,2016-09-06,Haemoglobin,139.0,September
5,3,2000-08-31,Platelets,250.0,August
6,3,2001-08-29,WCC,6.2,August


### Join second table with first to get see add a new postcode column

Now we can join the two original tables, `df` and `df2` to create a new table that has all the information in it, including the blood test results, dates and patient postcodes.

Remember that we need a column to join on; in this case we use the `patientid` column.

In [12]:
df.join(df2, left_on='patientid', right_on='patientid', how='left')

,patientid,date,blood_test_name,blood_test_result,postcode
0,1,2013-09-24,Haemoglobin,154.0,3000
1,1,2013-11-15,Haemoglobin,149.0,3000
2,1,2014-06-06,Platelets,226.0,3000
3,2,2016-09-01,Haemoglobin,132.0,3078
4,2,2016-09-06,Haemoglobin,139.0,3078
5,3,2000-08-31,Platelets,250.0,3056
6,3,2001-08-29,WCC,6.2,3056


## Combining multiple data transformation operations

Now we understand each of the operations individually, we can combine them in a sequence of transformations.

Below are a couple of exercises. The answers are shown at the end of the notebook.

### Exercise one

Get the maximum blood test result for each blood test, then sort from smallest to largest

In [ ]:
# include your response here
(
    df
    . # code for the groupby operation
    . # code for the sort operation
)

### Exercise two

Filter the data to haemoglobin results, then sort by patientid in reverse order, select the blood test name and result columns and then add a new column that labels the rows

In [ ]:
# include your response here
(
    df
    . # code for filtering
    . # code for sorting
    . # code for selecting the blood test name
    . # code that creates a new column 'row' with values [1, 2, 3, 4]
)

## Answers to exercises

### Exercise one

In [ ]:
(
    df
    .groupby('blood_test_name', 'blood_test_result', 'mean')
    .sort('blood_test_result', 'ascending')
)

### Exercise two

In [ ]:
(
    df
    .filter('blood_test_name', 'equals', 'Haemoglobin')
    .sort('patientid', 'ascending')
    .select(['blood_test_name', 'blood_test_result'])
    .add_column('row_number', [1,2,3,4])
)